In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing
from itertools import combinations
import gc
from sklearn.decomposition import PCA

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
median_vol = pd.read_csv("archive/MedianVolV2.csv")
median_vol.index.name = "stock_id";
median_vol = median_vol[['overall_medvol', "first5min_medvol", "last5min_medvol"]]
median_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
std_sizes = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()

In [7]:
def feat_eng(df):
    
    cols = [c for c in df.columns if c not in ['row_id', 'time_id']]
    df = df[cols]
    df = df.merge(median_vol, how = "left", left_on = "stock_id", right_index = True)
    
    df['bid_plus_ask_sizes'] = df['bid_size'] + train['ask_size']
#     df['median_size'] = df['stock_id'].map(median_sizes.to_dict())
    df['std_size'] = df['stock_id'].map(std_sizes.to_dict())
#     df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_size'], 1, 0) 
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for c in combinations(prices, 2):
        
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):
        
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)
    
        
    df.drop(columns=[
        # 'date_id', 
        'reference_price_far_price_imb',
        'reference_price_minus_near_price',
        'reference_price_near_price_imb',
        'far_price_near_price_imb',
        'far_price_ask_price_imb',
        'far_price_bid_price_imb',
        'far_price_minus_wap',
        'std_size',
        'bid_size_over_ask_size',
        'ask_price_bid_price_imb',
        'near_price_times_wap'
    ], inplace=True)
        
    gc.collect()

    df.replace([np.inf, -np.inf], 0, inplace=True)
    
    return df

In [8]:
y = train['target'].values
X = feat_eng(train)
prices = [c for c in train.columns if 'price' in c]
pca_prices = PCA(n_components=1)
X['pca_prices'] = pca_prices.fit_transform(X[prices].fillna(1))

In [9]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(X)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:13<01:50,  1.64it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [01:26<01:09,  1.43it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:52<00:44,  1.56it/s]

...

 79%|███████▉  | 158/200 [02:23<00:27,  1.56it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [03:00<00:00,  1.11it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<?, ?it/s]


In [10]:
train = []

In [11]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [9]:
model = torch_classes.GRUNetV2(12,64,num_layers=2).to(device='cuda:0')

In [10]:
model

GRUNetV2(
  (gru): GRU(12, 64, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((12800,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=12800, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=640, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [21]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(80,config['hidden_size'],num_layers=config['num_layers']).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        print(model)
        trading_df.reset_hidden(config['hidden_size'],num_layers=config['num_layers'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [13]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [14]:
config_static = {'learning_rate':0.0001, 'hidden_size':63, 'num_layers':2}

In [16]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]]
X = trading_data.packed_val_x[0] 
Y = trading_data.packed_val_y[0].data

In [17]:
# hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [18]:
# hidden_in 

In [19]:
# output,hidden = model(X,hidden_in,p1=True)

In [20]:
# model = torch_classes.GRUNetV2(12,64)
# model_loc = f"C:/Users/Nick/Documents/GitHub/OptiverKaggle/models/tough-totem-105/tough-totem-105_160.pt"
# model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
# print(model_data['model_state_dict'].keys())

# print(model_data['model_state_dict'].keys())
# del_keys = ['fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias']
# [model_data['model_state_dict'].pop(k) for k in del_keys]
# model.load_state_dict(model_data['model_state_dict'], strict=False)
# model = model.to(device='cuda:0')

In [22]:
model = model_pipeline(trading_data, config_static)


GRUNetV2(
  (gru): GRU(80, 63, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (batch_norm): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): LayerNorm((12600,), eps=1e-05, elementwise_affine=True)
  (fc0): Linear(in_features=12600, out_features=1024, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

{'epoch_loss': tensor(1.2448, device='cuda:0'), 'epoch_l1_loss': tensor(1.3579, device='cuda:0'), 'epoch': 0}
   stock  day  time    target      pred
0      0    0     0 -3.029704 -0.035211
1      1    0     0 -5.519986  0.032717
2      2    0     0 -8.389950  0.005464
3      3    0     0 -4.010201 -0.056723
4      4    0     0 -7.349849 -0.019788
5      5    0     0  6.779432 -0.004009
6      6    0     0 -2.499819  0.018939
7      7    0     0 -1.959801  0.006730
8      8    0     0 -5.970001 -0.032274
9      9    0     0  7.970333 -0.045781
created path
{'epoch_loss': tensor(1.2446, device='cuda:0'), 'epoch_l1_loss': tensor(1.3578, device='cuda:0'), 'epoch': 1}
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3578, device='cuda:0'), 'epoch': 2}
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3578, device='cuda:0'), 'epoch': 3}
{'epoch_loss': tensor(1.2445, device='cuda:0'), 'epoch_l1_loss': tensor(1.3578, device='cuda:0'), 'epoch': 

In [ ]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=0)

In [ ]:
stocks_hidden = [x.to('cpu') for x in stocks_hidden]

In [ ]:
stocks_hidden[0].shape

torch.Size([55, 63])

In [ ]:
len(stocks_hidden)

200

In [ ]:
torch.cat(stocks_hidden,dim=0).shape

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 63 but got size 64 for tensor number 69 in the list.